In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("100_Unique_QA_Dataset.csv")
df.head()

question      answer
0                   What is the capital of France?       Paris
1                  What is the capital of Germany?      Berlin
2               Who wrote 'To Kill a Mockingbird'?  Harper-Lee
3  What is the largest planet in our solar system?     Jupiter
4   What is the boiling point of water in Celsius?         100

In [3]:
def tokenize(text):
    text=text.lower()
    text=text.replace("?","")
    text=text.replace("'","")
    return text.split()


In [4]:
vocab={"<UOK>":0}

In [5]:
def build_vocab(row):
    tokenized_question=tokenize(row["question"])
    tokenized_answer=tokenize(row["answer"])
    merged_data=tokenized_question+tokenized_answer
    for token in merged_data:
        if token not in vocab:
            vocab[token]=len(vocab)
                             
    

In [6]:
df.apply(build_vocab,axis=1)

0     None
1     None
2     None
3     None
4     None
      ... 
85    None
86    None
87    None
88    None
89    None
Length: 90, dtype: object

In [7]:
vocab

{'<UOK>': 0,
 'what': 1,
 'is': 2,
 'the': 3,
 'capital': 4,
 'of': 5,
 'france': 6,
 'paris': 7,
 'germany': 8,
 'berlin': 9,
 'who': 10,
 'wrote': 11,
 'to': 12,
 'kill': 13,
 'a': 14,
 'mockingbird': 15,
 'harper-lee': 16,
 'largest': 17,
 'planet': 18,
 'in': 19,
 'our': 20,
 'solar': 21,
 'system': 22,
 'jupiter': 23,
 'boiling': 24,
 'point': 25,
 'water': 26,
 'celsius': 27,
 '100': 28,
 'painted': 29,
 'mona': 30,
 'lisa': 31,
 'leonardo-da-vinci': 32,
 'square': 33,
 'root': 34,
 '64': 35,
 '8': 36,
 'chemical': 37,
 'symbol': 38,
 'for': 39,
 'gold': 40,
 'au': 41,
 'which': 42,
 'year': 43,
 'did': 44,
 'world': 45,
 'war': 46,
 'ii': 47,
 'end': 48,
 '1945': 49,
 'longest': 50,
 'river': 51,
 'nile': 52,
 'japan': 53,
 'tokyo': 54,
 'developed': 55,
 'theory': 56,
 'relativity': 57,
 'albert-einstein': 58,
 'freezing': 59,
 'fahrenheit': 60,
 '32': 61,
 'known': 62,
 'as': 63,
 'red': 64,
 'mars': 65,
 'author': 66,
 '1984': 67,
 'george-orwell': 68,
 'currency': 69,
 'unit

In [8]:
len(vocab)

324

In [9]:
def word_to_index(text,vocab):
    indexed_text=[]
    for token in tokenize(text):
        if token in vocab:
            indexed_text.append(vocab[token])
        else:
            indexed_text.append(vocab["<UOK>"])
    return indexed_text

In [10]:
import torch
from torch.utils.data import Dataset,DataLoader

In [11]:
class CustomData(Dataset):
    def __init__(self,df,vocab):
        self.df=df
        self.vocab=vocab
    def __len__(self):
        return self.df.shape[0]
    def __getitem__(self,index):
        numerical_question=word_to_index(self.df.iloc[index]["question"],self.vocab)
        numerical_answer=word_to_index(self.df.iloc[index]["answer"],self.vocab)
        return torch.tensor(numerical_question),torch.tensor(numerical_answer)


In [12]:
dataset=CustomData(df,vocab)

In [13]:
dataloader=DataLoader(dataset,batch_size=1,shuffle=True)

In [14]:
for question,answer in dataloader:
    print(question,answer)

tensor([[  1,   2,   3, 122, 123,  19,   3,  45]]) tensor([[124]])
tensor([[ 1,  2,  3, 33, 34,  5, 35]]) tensor([[36]])
tensor([[10, 75, 76]]) tensor([[77]])
tensor([[  1,   2,   3, 180, 181, 182, 183]]) tensor([[184]])
tensor([[ 78,  79, 150, 151,  14, 152, 153]]) tensor([[154]])
tensor([[ 1,  2,  3, 50, 51, 19,  3, 45]]) tensor([[52]])
tensor([[42, 18,  2, 62, 63,  3, 64, 18]]) tensor([[65]])
tensor([[ 42, 137, 118,   3, 247,   5, 248]]) tensor([[249]])
tensor([[42, 43, 44, 45, 46, 47, 48]]) tensor([[49]])
tensor([[  1,   2,   3,  37,  38,  39, 161]]) tensor([[162]])
tensor([[10, 55,  3, 56,  5, 57]]) tensor([[58]])
tensor([[ 42,  86,  87, 241, 242,  19,  39, 243]]) tensor([[244]])
tensor([[ 42, 137,   2,  62,  39,   3, 322, 323]]) tensor([[6]])
tensor([[ 42, 137,   2, 138,  39, 139]]) tensor([[53]])
tensor([[78, 79, 80, 81, 82, 83, 84]]) tensor([[85]])
tensor([[ 42, 167,   2,   3,  17, 168, 169]]) tensor([[170]])
tensor([[ 10,  29, 130, 131]]) tensor([[132]])
tensor([[ 42,   2,   3

In [36]:
import torch.nn as nn

In [37]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [38]:
learning_rate=0.01
epochs=20

In [42]:
model=SimpleRNN(len(vocab))

In [44]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [45]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 541.986753
Epoch: 2, Loss: 349.263960
Epoch: 3, Loss: 156.768204
Epoch: 4, Loss: 56.468060
Epoch: 5, Loss: 57.353689
Epoch: 6, Loss: 33.272257
Epoch: 7, Loss: 22.596772
Epoch: 8, Loss: 17.265087
Epoch: 9, Loss: 19.426494
Epoch: 10, Loss: 10.411605
Epoch: 11, Loss: 1.538152
Epoch: 12, Loss: 0.873577
Epoch: 13, Loss: 0.695252
Epoch: 14, Loss: 0.583560
Epoch: 15, Loss: 0.509193
Epoch: 16, Loss: 0.450470
Epoch: 17, Loss: 0.403094
Epoch: 18, Loss: 0.363486
Epoch: 19, Loss: 0.330749
Epoch: 20, Loss: 0.302764


In [53]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = word_to_index(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")
  else:
    print(list(vocab.keys())[index])

In [55]:
predict(model," by")

I don't know
